In [1]:
# make sure run prepare-mixed-stt-train.ipynb first
# make sure run prepare-malay-stt-test.ipynb first

In [2]:
import os
from glob import glob
from tqdm import tqdm
import json

In [3]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [4]:
with open('singlish-mixed.json') as fopen:
    data = json.load(fopen)

In [5]:
audios = data['test']
audios, texts = zip(*audios)

In [6]:
processed_text = [preprocessing_text(t) for t in texts]

In [7]:
with open('bahasa-asr-test.json') as fopen:
    bahasa = json.load(fopen)
    
bahasa['X'][:1]

['iium/audio-iium/7.wav']

In [8]:
audios = bahasa['X'] + list(audios)
processed_text = bahasa['Y'] + list(processed_text)

In [9]:
with open('mixed-asr-test.json', 'w') as fopen:
    json.dump({'X': audios, 'Y':processed_text}, fopen)

In [ ]:
# import malaya_speech

In [ ]:
# tokenizer = malaya_speech.subword.load('transducer.subword')
# tokenizer

In [ ]:
# malaya_speech.subword.decode(tokenizer, [0, 2, 133, 875])

In [ ]:
# from pydub import AudioSegment
# import numpy as np

# sr = 16000

# def mp3_to_wav(file, sr = sr):
#     audio = AudioSegment.from_file(file)
#     audio = audio.set_frame_rate(sr).set_channels(1)
#     sample = np.array(audio.get_array_of_samples())
#     return malaya_speech.astype.int_to_float(sample), sr

# def generator(maxlen = 18, min_length_text = 2):
#     for i in tqdm(range(len(audios))):
#         try:
#             if audios[i].endswith('.mp3'):
#                 wav_data, _ = mp3_to_wav(audios[i])
#             else:
#                 wav_data, _ = malaya_speech.load(audios[i])
                
#             if (len(wav_data) / sr) > maxlen:
#                 print(f'skipped audio too long {audios[i]}')
#                 continue
                
#             if len(processed_text[i]) < min_length_text:
#                 print(f'skipped text too short {audios[i]}')
#                 continue    

#             yield {
#                 'waveforms': wav_data.tolist(),
#                 'waveform_lens': [len(wav_data)],
#                 'targets': malaya_speech.subword.encode(tokenizer, processed_text[i], add_blank = False),
#             }
#         except Exception as e:
#             print(e)
            
# generator = generator()

In [ ]:
# import os
# import tensorflow as tf

# os.system('rm bahasa-asr-test/data/*')
# DATA_DIR = os.path.expanduser('bahasa-asr-test/data')
# tf.gfile.MakeDirs(DATA_DIR)

In [ ]:
# shards = [{'split': 'dev', 'shards': 10}]

In [ ]:
# import malaya_speech.train as train

# train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')